# data는 김도휘 형제님과 김명찬 형제님이 만들어주신 보편지향 기도 데이터를 사용하였습니다. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## CSV 에서 기도문 읽어오기
def read_data(path_to_file):
    df = pd.read_csv(path_to_file, dtype=str)
    return df

df = read_data('../../data/pray456_v3.csv')

In [2]:
df.to_csv('../../data/pray456_v3withid.csv')

In [3]:
X = df['content']
y = df['label']
print(len(X))
print(len(y))

774
774


In [4]:
X[0]

'주님, 대림시기를 맞는 교회가 회개와 화해의 생활을 하며 저희에게  오실 아기 예수님을 기쁜 마음으로 맞이할 수 있도록 도와주소서.'

In [5]:
y_quiz = df['content'].sample(50)
y_quiz.shape

(50,)

In [6]:
y_quiz.sort_index().to_csv('../../data/quiz_pray1_sample50.csv')

## y data one_hot encoding

In [7]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

print(type(y[0]), y[:5])

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded[:5])

# # one_hot encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), )
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded[:5])

# setup y 
# y = onehot_encoded
y= integer_encoded
print(y[:5])

<class 'str'> 0    1
1    2
2    3
3    4
4    1
Name: label, dtype: object
[0 1 2 3 0]
[0 1 2 3 0]


## 띄어쓰기로 구분

In [8]:
X = [x.split() for x in X]

In [9]:
X[0]

['주님,',
 '대림시기를',
 '맞는',
 '교회가',
 '회개와',
 '화해의',
 '생활을',
 '하며',
 '저희에게',
 '오실',
 '아기',
 '예수님을',
 '기쁜',
 '마음으로',
 '맞이할',
 '수',
 '있도록',
 '도와주소서.']

## 고유 토큰 인덱싱

In [10]:
from collections import defaultdict

In [11]:
# 단어마다 고유한 인덱스를 부여하기 위한 dictionary
token_to_index = defaultdict(lambda : len(token_to_index))

In [12]:
# 단어에 대한 고유 인덱스를 부여하는 함수
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token_to_index[token] for token in tokens]
    return

In [13]:
X = list(convert_token_to_idx(X))

In [14]:
# 고유 인덱스로 변환될 경우, 원래 어떤 단어였는지 알기 어려우므로,
# 인덱스로 변환된 단어를 본래의 단어로 재변환하기 위한 dictionary 생성
index_to_token = {val : key for key,val in token_to_index.items()}

#### 인덱싱 결과 확인 

In [15]:
import operator

In [16]:
for k,v in sorted(token_to_index.items(), key=operator.itemgetter(1))[:5]:
    print (k,v)

주님, 0
대림시기를 1
맞는 2
교회가 3
회개와 4


In [17]:
X[0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

### 빈(empty) 단어 가방(Bag of Words) 생성

In [18]:
n_train_reviews = len(X)       # 학습용 리뷰의 총 수
n_unique_word = len(token_to_index)  # 고유 단어의 갯수 (BOW의 차원의 크기) 

In [19]:
n_unique_word

3329

### numpy를 사용하면 memory error 발생 

In [20]:
import numpy as np

In [21]:
bow = np.zeros((n_train_reviews, n_unique_word), dtype=np.float32)

### Scipy 패키지 활용

In [22]:
# import scipy.sparse as sps

In [23]:
# 학습용 리뷰 수(150,000) x 고유 단어의 수(450,541)의 크기를 갖는 빈 단어가방 생성
# bow_data = sps.lil_matrix((n_train_reviews, n_unique_word), dtype=np.int8)

### 단어 가방 채우기

In [24]:
for i, tokens in enumerate(X):
    for token in tokens:
        # i번 째 리뷰에 등장한 단어들을 세서, 고유 번호에 1씩 더해준다.
        bow[i, token] += 1.0

### Train / test split

In [25]:
bow_train, bow_test, y_train, y_test = train_test_split(bow, y, test_size=0.2, random_state=1212)
print(bow_train.shape, bow_test.shape, y_train.shape, y_test.shape)
print(y_train[:5])

(619, 3329) (155, 3329) (619,) (155,)
[3 2 3 1 0]


## Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

### Train

In [28]:
model.fit(bow_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

### Test

In [29]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [30]:
predict = model.predict(bow_test)
accuracy = accuracy_score(y_test, predict)

In [31]:
print('Accuracy : ',accuracy)
print(classification_report(y_test, predict))

Accuracy :  0.7870967741935484
             precision    recall  f1-score   support

          0       0.86      0.95      0.90        38
          1       0.84      0.76      0.80        42
          2       0.81      0.71      0.75        41
          3       0.64      0.74      0.68        34

avg / total       0.79      0.79      0.79       155



## Pytorch

In [32]:
import torch

In [33]:
# dataset : bow_train, bow_test, y_train, y_test
bow_train, y_train, bow_test, y_test = map(
    torch.tensor, (bow_train, y_train, bow_test, y_test)
)

In [34]:
n, c = bow_train.shape
print(bow_train.shape)

torch.Size([619, 3329])


In [35]:
print(y_train.min(), y_train.max())
print(y_test.min(), y_test.max())

tensor(0) tensor(3)
tensor(0) tensor(3)


In [36]:
bs = 64  # batch size

xb = bow_train[0:bs]  # a mini-batch from x
yb = y_train[0:bs]
xv = bow_test[0:bs]
yv = y_test[0:bs]

print(xb[:5])
print(yb[:5])


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([3, 2, 3, 1, 0])


### Logistic regression with equations

In [37]:
import math

weights = torch.randn(3329, 4) / math.sqrt(3329)
weights.requires_grad_()
bias = torch.zeros(4, requires_grad=True)

def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

def nll(pred, gt):
    return -pred[range(gt.shape[0]), gt].mean()

loss_func = nll

def accuracy(out, y):
    preds = torch.argmax(out, dim=1)
    return (preds == y).float().mean()

print(loss_func(model(xb), yb), accuracy(model(xb), yb))
print(loss_func(model(xv), yv), accuracy(model(xv), yv))


tensor(1.3900, grad_fn=<NegBackward>) tensor(0.2188)
tensor(1.3963, grad_fn=<NegBackward>) tensor(0.1875)


In [38]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 30  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        
        start_i = i * bs
        end_i = start_i + bs
        xb = bow_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        
        ## Feed foward
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        ## Backpropagation
        loss.backward()
        
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
    
    print("Epoch:", epoch, loss_func(model(xb), yb), accuracy(model(xb), yb), loss_func(model(xv), yv), accuracy(model(xv), yv))

Epoch: 0 tensor(1.0815, grad_fn=<NegBackward>) tensor(0.8372) tensor(1.1885, grad_fn=<NegBackward>) tensor(0.5312)
Epoch: 1 tensor(0.9152, grad_fn=<NegBackward>) tensor(0.8605) tensor(1.0386, grad_fn=<NegBackward>) tensor(0.6562)
Epoch: 2 tensor(0.7978, grad_fn=<NegBackward>) tensor(0.8837) tensor(0.9409, grad_fn=<NegBackward>) tensor(0.7188)
Epoch: 3 tensor(0.7085, grad_fn=<NegBackward>) tensor(0.9070) tensor(0.8722, grad_fn=<NegBackward>) tensor(0.7656)
Epoch: 4 tensor(0.6372, grad_fn=<NegBackward>) tensor(0.9070) tensor(0.8213, grad_fn=<NegBackward>) tensor(0.7969)
Epoch: 5 tensor(0.5785, grad_fn=<NegBackward>) tensor(0.9302) tensor(0.7822, grad_fn=<NegBackward>) tensor(0.8125)
Epoch: 6 tensor(0.5292, grad_fn=<NegBackward>) tensor(0.9302) tensor(0.7512, grad_fn=<NegBackward>) tensor(0.8125)
Epoch: 7 tensor(0.4870, grad_fn=<NegBackward>) tensor(0.9302) tensor(0.7261, grad_fn=<NegBackward>) tensor(0.7969)
Epoch: 8 tensor(0.4506, grad_fn=<NegBackward>) tensor(0.9535) tensor(0.7055, gra

In [39]:
xb

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [40]:
yb

tensor([1, 2, 1, 1, 0, 3, 3, 3, 0, 3, 0, 3, 1, 3, 2, 1, 0, 3, 1, 0, 3, 0, 1, 3,
        3, 2, 3, 3, 0, 1, 3, 1, 0, 3, 0, 0, 1, 1, 3, 1, 1, 1, 0])

In [41]:
## What is nn.Module?

In [43]:
## Same model with nn.Module
from torch import nn

class Logistic_Regression(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(3329, 4) / math.sqrt(3329))
        self.bias = nn.Parameter(torch.zeros(4))

    def forward(self, xb):
        return log_softmax(xb @ self.weights + self.bias)

model = Logistic_Regression()
loss_func = nll

with torch.no_grad(): # To see model's paramter without calculating gradient
    for p in model.parameters(): 
        print(p)
    print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Parameter containing:
tensor([[-0.0246, -0.0038,  0.0053, -0.0121],
        [ 0.0251,  0.0316,  0.0260, -0.0077],
        [ 0.0009, -0.0002, -0.0086, -0.0164],
        ...,
        [ 0.0015, -0.0057,  0.0010, -0.0057],
        [ 0.0131, -0.0044,  0.0278, -0.0196],
        [ 0.0123,  0.0283,  0.0307, -0.0147]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)
tensor(1.3787) tensor(0.3256)


In [44]:
def fit():
    for epoch in range(epochs):
        n_batches = (n - 1) // bs + 1
        for i in range(n_batches):
            start_i = i * bs
            end_i = start_i + bs
            
            xb = bow_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            
            pred = model(xb)
            loss = loss_func(pred, yb)
            
            #back propagation
            loss.backward()
            
            # update weights
            with torch.no_grad():
                for p in model.parameters():
                     p -= p.grad * lr
                model.zero_grad()
        
        print("Epochs:", epoch, loss_func(model(xb), yb), accuracy(model(xb), yb), loss_func(model(xv), yv), accuracy(model(xv), yv))

fit()

Epochs: 0 tensor(1.0662, grad_fn=<NegBackward>) tensor(0.7907) tensor(1.1656, grad_fn=<NegBackward>) tensor(0.6406)
Epochs: 1 tensor(0.9029, grad_fn=<NegBackward>) tensor(0.8605) tensor(1.0187, grad_fn=<NegBackward>) tensor(0.6875)
Epochs: 2 tensor(0.7876, grad_fn=<NegBackward>) tensor(0.9070) tensor(0.9231, grad_fn=<NegBackward>) tensor(0.7188)
Epochs: 3 tensor(0.6997, grad_fn=<NegBackward>) tensor(0.9070) tensor(0.8562, grad_fn=<NegBackward>) tensor(0.7812)
Epochs: 4 tensor(0.6295, grad_fn=<NegBackward>) tensor(0.9070) tensor(0.8069, grad_fn=<NegBackward>) tensor(0.7969)
Epochs: 5 tensor(0.5717, grad_fn=<NegBackward>) tensor(0.9302) tensor(0.7690, grad_fn=<NegBackward>) tensor(0.7969)
Epochs: 6 tensor(0.5231, grad_fn=<NegBackward>) tensor(0.9302) tensor(0.7390, grad_fn=<NegBackward>) tensor(0.7969)
Epochs: 7 tensor(0.4815, grad_fn=<NegBackward>) tensor(0.9535) tensor(0.7149, grad_fn=<NegBackward>) tensor(0.7969)
Epochs: 8 tensor(0.4455, grad_fn=<NegBackward>) tensor(0.9767) tensor(0.

## Train with Pytorch CrossEntrophy

In [51]:
from torch import nn
import torch.nn.functional as F

loss_func = F.cross_entropy

class Logistic_Regression(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(3329, 4) / math.sqrt(3329))
        self.bias = nn.Parameter(torch.zeros(4))

    def forward(self, xb):
        return xb @ self.weights + self.bias

model = Logistic_Regression()

with torch.no_grad(): # To see model's paramter without calculating gradient
    for p in model.parameters(): 
        print(p)
    print(model(xb).shape)
    print(yb.shape)
    
    print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Parameter containing:
tensor([[ 0.0498, -0.0229,  0.0058, -0.0068],
        [ 0.0021,  0.0212, -0.0076,  0.0046],
        [ 0.0232, -0.0356,  0.0176,  0.0359],
        ...,
        [ 0.0418, -0.0022, -0.0249, -0.0178],
        [ 0.0153, -0.0133,  0.0092,  0.0018],
        [-0.0436,  0.0283, -0.0374,  0.0161]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)
torch.Size([43, 4])
torch.Size([43])
tensor(1.3832) tensor(0.2558)


In [52]:
loss = loss_func(model(xb), yb)
loss.backward()

In [53]:
loss_func

<function torch.nn.functional.cross_entropy>

In [50]:
fit()

Epochs: 0 tensor(1.0779, grad_fn=<NllLossBackward>) tensor(0.8140) tensor(1.1801, grad_fn=<NllLossBackward>) tensor(0.6875)
Epochs: 1 tensor(0.9139, grad_fn=<NllLossBackward>) tensor(0.8605) tensor(1.0327, grad_fn=<NllLossBackward>) tensor(0.7344)
Epochs: 2 tensor(0.7977, grad_fn=<NllLossBackward>) tensor(0.8837) tensor(0.9365, grad_fn=<NllLossBackward>) tensor(0.7188)
Epochs: 3 tensor(0.7090, grad_fn=<NllLossBackward>) tensor(0.9070) tensor(0.8690, grad_fn=<NllLossBackward>) tensor(0.7812)
Epochs: 4 tensor(0.6381, grad_fn=<NllLossBackward>) tensor(0.9302) tensor(0.8191, grad_fn=<NllLossBackward>) tensor(0.8125)
Epochs: 5 tensor(0.5796, grad_fn=<NllLossBackward>) tensor(0.9302) tensor(0.7807, grad_fn=<NllLossBackward>) tensor(0.7969)
Epochs: 6 tensor(0.5303, grad_fn=<NllLossBackward>) tensor(0.9302) tensor(0.7504, grad_fn=<NllLossBackward>) tensor(0.7969)
Epochs: 7 tensor(0.4881, grad_fn=<NllLossBackward>) tensor(0.9302) tensor(0.7259, grad_fn=<NllLossBackward>) tensor(0.7969)
Epochs: 

In [76]:
from torch import nn, optim
import torch.nn.functional as F

loss_func = F.cross_entropy

class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(3329,4)
        self.softmax = torch.nn.Softmax() # instead of Heaviside step fn
        
    def forward(self, x):
        output = self.fc(x)
        output = self.softmax(x) # instead of Heaviside step fn
        return output

model = Perceptron()

with torch.no_grad(): # To see model's paramter without calculating gradient
    for p in model.parameters(): 
        print(p)
    print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Parameter containing:
tensor([[-1.3385e-02,  8.2254e-03, -6.3626e-03,  ...,  3.2154e-03,
         -1.5589e-02, -4.7928e-03],
        [-1.3171e-02,  1.7026e-02, -4.5189e-03,  ...,  9.7757e-03,
         -3.4634e-04, -1.3953e-02],
        [ 9.7065e-03, -1.5890e-02,  9.7550e-03,  ...,  1.2983e-02,
         -6.8674e-05,  1.1500e-02],
        [ 6.4180e-03, -1.3813e-03,  1.2486e-02,  ..., -6.7461e-03,
         -7.7586e-03, -2.1890e-03]], requires_grad=True)
Parameter containing:
tensor([-0.0074, -0.0015, -0.0074, -0.0038], requires_grad=True)
tensor(8.1103) tensor(0.)


C:\Users\kmc55\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [77]:
loss = loss_func(model(xb), yb)
loss.backward()

C:\Users\kmc55\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [78]:
criterion = F.cross_entropy
optimizer = optim.SGD(model.parameters(), lr = 0.01)

model.train()
epoch = 20

for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(bow_train)
    # Compute Loss
    loss = criterion(y_pred, y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 8.110262870788574


C:\Users\kmc55\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn